In [1]:
import tensorflow as tf

In [2]:
train_data_path = "file:///C:/SourceCode/GitHub/wayneg/recs-models/data/sampledata/modelSamples.csv"
sample_file_path = tf.keras.utils.get_file("modelSamples.csv", train_data_path)

In [3]:
def make_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True
    )

    return dataset

In [4]:
raw_samples_data = make_dataset(sample_file_path)
print(raw_samples_data)

<PrefetchDataset shapes: (OrderedDict([(movieId, (None,)), (userId, (None,)), (rating, (None,)), (timestamp, (None,)), (releaseYear, (None,)), (movieGenre1, (None,)), (movieGenre2, (None,)), (movieGenre3, (None,)), (movieRatingCount, (None,)), (movieAvgRating, (None,)), (movieRatingStddev, (None,)), (userRatedMovie1, (None,)), (userRatedMovie2, (None,)), (userRatedMovie3, (None,)), (userRatedMovie4, (None,)), (userRatedMovie5, (None,)), (userRatingCount, (None,)), (userAvgReleaseYear, (None,)), (userReleaseYearStddev, (None,)), (userAvgRating, (None,)), (userRatingStddev, (None,)), (userGenre1, (None,)), (userGenre2, (None,)), (userGenre3, (None,)), (userGenre4, (None,)), (userGenre5, (None,))]), (None,)), types: (OrderedDict([(movieId, tf.int32), (userId, tf.int32), (rating, tf.float32), (timestamp, tf.int32), (releaseYear, tf.int32), (movieGenre1, tf.string), (movieGenre2, tf.string), (movieGenre3, tf.string), (movieRatingCount, tf.int32), (movieAvgRating, tf.float32), (movieRatingSt

In [5]:
test_dataset = raw_samples_data.take(1000)
train_dataset = raw_samples_data.skip(1000)

In [6]:
movie_columns = []
# movie id embedding
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
movie_columns.append(movie_emb_col)

In [7]:
user_columns = []
# user id embedding
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
user_columns.append(user_emb_col)


In [8]:

inputs = {
    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32')
}

In [9]:
# model architecture
movie_deep = tf.keras.layers.DenseFeatures(movie_columns)(inputs)
movie_deep = tf.keras.layers.Dense(units=20, activation='relu')(movie_deep)
movie_deep = tf.keras.layers.Dense(units=20, activation='relu')(movie_deep)

user_deep = tf.keras.layers.DenseFeatures(user_columns)(inputs)
user_deep = tf.keras.layers.Dense(units=20, activation='relu')(user_deep)
user_deep = tf.keras.layers.Dense(units=20, activation='relu')(user_deep)

output_layer = tf.keras.layers.Dot(axes=1)([movie_deep, user_deep])

model = tf.keras.Model(inputs, output_layer)

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [11]:
model.fit(train_dataset, epochs=5)

Epoch 1/5


C:\Users\nju_g\.conda\envs\recs-learning\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['rating', 'timestamp', 'releaseYear', 'movieGenre1', 'movieGenre2', 'movieGenre3', 'movieRatingCount', 'movieAvgRating', 'movieRatingStddev', 'userRatedMovie1', 'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4', 'userRatedMovie5', 'userRatingCount', 'userAvgReleaseYear', 'userReleaseYearStddev', 'userAvgRating', 'userRatingStddev', 'userGenre1', 'userGenre2', 'userGenre3', 'userGenre4', 'userGenre5'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


8232/8232 [==============================] - 50s 6ms/step - loss: 0.6489 - accuracy: 0.6427
Epoch 2/5
8232/8232 [==============================] - 48s 6ms/step - loss: 0.6191 - accuracy: 0.6861
Epoch 3/5
8232/8232 [==============================] - 51s 6ms/step - loss: 0.5988 - accuracy: 0.7168
Epoch 4/5
8232/8232 [==============================] - 54s 7ms/step - loss: 0.6131 - accuracy: 0.7204
Epoch 5/5
8232/8232 [==============================] - 60s 7ms/step - loss: 0.5852 - accuracy: 0.7448


In [12]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

1000/1000 [==============================] - 4s 4ms/step - loss: 0.5967 - accuracy: 0.7060
Test Loss 0.5966747403144836, Test Accuracy 0.7059999704360962


In [13]:
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))

Predicted good rating: 22.64%  | Actual rating label:  Good Rating
Predicted good rating: 26.83%  | Actual rating label:  Bad Rating
Predicted good rating: 16.57%  | Actual rating label:  Good Rating
Predicted good rating: 15.58%  | Actual rating label:  Good Rating
Predicted good rating: 59.84%  | Actual rating label:  Good Rating
Predicted good rating: 20.76%  | Actual rating label:  Bad Rating
Predicted good rating: 80.13%  | Actual rating label:  Bad Rating
Predicted good rating: 30.41%  | Actual rating label:  Bad Rating
Predicted good rating: 35.53%  | Actual rating label:  Bad Rating
Predicted good rating: 29.31%  | Actual rating label:  Good Rating
Predicted good rating: 111.91%  | Actual rating label:  Bad Rating
Predicted good rating: 66.38%  | Actual rating label:  Good Rating
